In [1]:
import pandas as pd
import os
import ast
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from tqdm import tqdm
import time

In [2]:
base_dir = os.getcwd()
output_dir = os.path.join(base_dir, "..", "..", "output")

file_path1 = os.path.join(output_dir, "news_batch_0.csv")
df1 = pd.read_csv(file_path1)

In [3]:
def read_url(urls):
    urls = ast.literal_eval(urls) #Convert string to list

    if len(urls) == 0 or urls is None or urls == [] or urls == ['', '']:
        return "No URL"
    elif len(urls) == 2:
        return urls[0], urls[1]
    else:
        return urls[0]

In [4]:
def get_redirected_url(url):
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-extensions")
    options.add_argument("--disable-infobars")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--window-size=1920x1080")  # Set window size for headless mode

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    try:
        driver.set_page_load_timeout(10)  # Set a timeout for page load
        driver.get(url)
        final_url = driver.current_url
    except Exception as e:
        final_url = f"Error: {str(e)}"
    finally:
        driver.quit()

    return final_url

In [5]:
df1[['url1', 'url2']] = df1['news_urls'].apply(lambda x: read_url(x)).apply(pd.Series)

In [6]:
df1.head()

,Message_ID,Date,Cleaned_text,Query_Text,news_urls,url1,url2
0,1266,10/31/2024 23:59,donald trump hate america want eliminate israe...,donald trump hate america israel country donal...,"['https://apple.news/AdzGKUlFhQa2trdrknd9jvQ',...",https://apple.news/AdzGKUlFhQa2trdrknd9jvQ,https://apple.news/ATNDpVZ-QSRGFn_BzMp-5tA
1,1268,11/1/2024 6:06,breaking germany called european union classif...,germany european union revolutionary guard cor...,"['/search?num=4', 'https://t-s.news/r?u=aHR0cH...",/search?num=4,https://t-s.news/r?u=aHR0cHM6Ly93d3cudGhlZ3Vhc...
2,1269,11/1/2024 9:12,accurate photo right israel israel v hamas syr...,photo israel israel v hamas syrian israel israel,['https://t-s.news/r?u=aHR0cHM6Ly93d3cuYWxqYXp...,https://t-s.news/r?u=aHR0cHM6Ly93d3cuYWxqYXplZ...,https://apple.news/AKE1Nx-HZTce7voebcAYO1w
3,1273,11/1/2024 14:32,breaking idf released footage precise eliminat...,idf footage elimination hamas terrorist israel,['https://t-s.news/r?u=aHR0cHM6Ly93d3cuZm94bmV...,https://t-s.news/r?u=aHR0cHM6Ly93d3cuZm94bmV3c...,https://apple.news/AjXpFOJQtTVyYS-nPSJVdiA
4,1274,11/1/2024 18:50,love israel heart please comment shabbat shalo...,israel heart shabbat shalom unity strength adv...,['https://jewishlink.news/to-strengthen-and-be...,https://jewishlink.news/to-strengthen-and-be-s...,https://t-s.news/r?u=aHR0cHM6Ly93d3cuaGFhcmV0e...


In [7]:
tqdm.pandas()
df1['url1_'] = df1['url1'].progress_apply(lambda x: get_redirected_url(x)).apply(pd.Series)

100%|██████████| 1554/1554 [4:46:54<00:00, 11.08s/it]  


In [8]:
file_path = os.path.join(output_dir, "redirected_urls.csv")
df1.to_csv(file_path, index=False)

In [11]:
tqdm.pandas()
df1['url2_'] = df1['url2'].progress_apply(lambda x: get_redirected_url(x)).apply(pd.Series)

100%|██████████| 1554/1554 [5:15:13<00:00, 12.17s/it]  


In [12]:
file_path = os.path.join(output_dir, "redirected_urls_1.csv")
df1.to_csv(file_path, index=False)